---
title: How to submit a PwBaseWorkChain 
subtitle: 
#author:
#  - name: Miki Bonacci
    #affiliations: Executable Books; Curvenote
    #orcid: 0000-0002-7859-8394
#    email: miki.bonacci@psi.ch
license:
  code: MIT
#date: 2023/01/23
---

In the following we show how to run a calculation with the `aiida-quantumespresso` plugin using the new *atomistic* `StructureData`. 
The procedure is very similar to the [old way](https://aiida-quantumespresso.readthedocs.io/en/latest/tutorials/first_pw.html), except to the fact that now properties like magnetization and charge are provided within the structure
and not in the input parameters of the `PwCalculation`.

***For now, only magnetization is supported.***

***PLEASE USE THE FOLLOWING VERSIONS OF THE RELATED CODES:***

- aiida-atomistic: [develop branch](https://github.com/mikibonacci/aiida-atomistic/tree/develop)
- aiida-core: [new_StructureData branch](https://github.com/mikibonacci/aiida-core/tree/new_StructureData)
- aiida-quantumespresso: [new_StructureData branch](https://github.com/mikibonacci/aiida-quantumespresso/tree/new_StructureData)
- aiida-pseudo: [new_StructureData branch](https://github.com/mikibonacci/aiida-pseudo/tree/new_StructureData)

In [1]:
from aiida import load_profile, orm
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_atomistic import StructureData, StructureDataMutable

load_profile()

Profile<uuid='1a5a8d0836814a04a238c67cc7481655' name='default'>

Once loaded the necessary modules, we start building up our `StructureDataMutable`, to be then converted into the AiiDA `StructureData` (via the `to_immutable` method):

In [2]:
mutable_structure = StructureDataMutable()
mutable_structure.set_cell([[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]])

mutable_structure.add_atom({
            'symbol':'Si',
            'position':[1/2, 1/2, 1/2],
            'kind_name': 'Si1'
        })
mutable_structure.add_atom({
            'symbol':'Si',
            'position':[3/4, 3/4, 3/4],
            'kind_name': 'Si1'
        })

structure = mutable_structure.to_immutable()

We then generate the `PwBaseWorkChain` builder via the `get_builder_from_protocol` method, providing as input the *pw.x* code node and the structure:

In [3]:
from aiida.engine import submit, run_get_node


builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=structure,
    protocol="fast",
    )

run = run_get_node(builder)

09/15/2024 09:37:04 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1593|PwBaseWorkChain|run_process]: launching PwCalculation<1598> iteration #1
09/15/2024 09:37:07 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1593|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/15/2024 09:37:07 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1593|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


After the job is finished, you can inspect the outputs as usual:

In [4]:
run[0]

{'remote_folder': <RemoteData: uuid: 5ab0d114-a2bb-4b52-8ec8-7b24d4165661 (pk: 1599)>,
 'retrieved': <FolderData: uuid: b6049d24-4506-481a-ab34-301cfc3f721f (pk: 1600)>,
 'output_parameters': <Dict: uuid: e68b3d92-5447-41d7-a98c-ca1890926775 (pk: 1603)>,
 'output_trajectory': <TrajectoryData: uuid: 34541767-833a-4548-a0d0-25cedc7b5d48 (pk: 1602)>,
 'output_band': <BandsData: uuid: 18b18c7d-9dce-4196-b947-d0277bbb673f (pk: 1601)>}

## How to deal with magnetic configurations

The first way to prepare a magnetic calculation is to trigger the `nspin` or `noncolin` in the input parameters of the `PwCalculation` respectively for *collinear* (`nspin=2`) and *non collinear* (`noncolin=.True.`) calculations.

Of course, the magnetic moments have to be provided within the `StructureData`, so that they can be used to build the [*starting_magnetization*](https://www.quantum-espresso.org/Doc/INPUT_PW.html#idm301) variable in the `&SYSTEM` namelist.


### Collinear case 

We first load the structure using Pymatgen to parse the *mcif* file for BCC Iron:

In [5]:
from pymatgen.core import Structure

#suppose you are in the `aiida-atomistic` root folder
iron_bcc = Structure.from_file('../examples/structure/data/Fe_bcc.mcif', primitive=True)
magnetic_structure = StructureData.from_pymatgen(iron_bcc)

print(magnetic_structure.properties.magmoms)

[[0.0, 0.0, 2.5]]


In [6]:
magnetic_structure.properties.kinds

['Fe0']

We can check that our structure is collinear (trivial in this case):

In [7]:
magnetic_structure.is_collinear

True

For now, we consider only collinear magnetic moments along the z axis.

We can create again the builder as before, this time overriding `nspin` to be equal to 2 (*collinear* calculation):

In [8]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"nspin": 2}
            }
        }
    }
    )

In [9]:
run = run_get_node(builder)

09/15/2024 09:37:10 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1610|PwBaseWorkChain|run_process]: launching PwCalculation<1615> iteration #1
09/15/2024 09:37:18 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1610|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/15/2024 09:37:18 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1610|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


### The output magnetic moments

For now, the new output magnetization of a structure will not be stored in the output `StructureData` (if any). 
The reason is that the output magnetization can present some discrepancies which are actually artifacts of the simulation. For example, symmetries maybe not correctly detected (due to different kinds) and so magnetization magnitudes (and orientations) are not the same as expected. 

So, the if the user needs it, he can use a `calcfuntion` to generate a new `StructureData` with the new magnetic moments (both as list of *floats* or of len=3 *lists*):

In [10]:
trajectory = run[-1].called[-1].outputs.output_trajectory # run[1].called[-1].outputs.output_trajectory
magnetic_moments = [magmom[0] for magmom in trajectory.get_array('atomic_magnetic_moments')]
print(magnetic_moments)

[2.3254]


In [11]:
from aiida_quantumespresso.utils.magnetic import generate_structure_with_magmoms

new_structure = generate_structure_with_magmoms(magnetic_structure, magnetic_moments)
print(f"New StructureData: {new_structure}\nAttached magmoms: {new_structure.properties.magmoms}")

New StructureData: uuid: d8752863-b018-4fa0-a108-70863e2f29de (pk: 1623)
Attached magmoms: [[0.0, 0.0, 2.3254]]


Using this utility function, kinds will be automatically detected. 

If you don't want the provenance and the new `StructureData` node to be stored, add `metadata={"store_provenance": False}` to the input of the `calcfunction`. 

### Non-collinear case 

For the non-collinear case, we just need to put `noncolin: True` in the input parameters:

In [12]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"noncolin": True}
            }
        }
    }
    )

In [13]:
run = run_get_node(builder)

09/15/2024 09:37:19 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1629|PwBaseWorkChain|run_process]: launching PwCalculation<1634> iteration #1
09/15/2024 09:37:37 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1629|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/15/2024 09:37:37 AM <3027679> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1629|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [14]:
run[0]

{'remote_folder': <RemoteData: uuid: cc1c05cf-12e4-4871-a7fd-e533edc0ebfb (pk: 1635)>,
 'retrieved': <FolderData: uuid: 458eeee8-5ff2-42e5-bd6a-097b7299ae52 (pk: 1636)>,
 'output_parameters': <Dict: uuid: a1a013b4-9747-48ec-aeba-02a93e57aaed (pk: 1639)>,
 'output_trajectory': <TrajectoryData: uuid: 1aaf7a83-d59e-4462-88fd-6e560bb2577d (pk: 1638)>,
 'output_band': <BandsData: uuid: 176e3e7e-2ce2-4f52-a880-b9dec84a9b74 (pk: 1637)>}

Currently, no parser for non-collinear magnetization is implemented in the `aiida-quantumespresso` plugin.